# Notebook Title

## Setup Python and R environment
you can ignore this section

In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [2]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


## Load & Clean Data

👉 Load the data along with the census connectors below (the output of the `connect-to-census.ipynb` notebook) and do any cleanup you'd like to do.

In [4]:
import pandas as pd

In [5]:
df_merge2.columns.to_list()

NameError: name 'df_merge2' is not defined

In [ ]:
df_merge2 = pd.read_csv('schoolData.csv', dtype={'TRACT': str, 'COUNTY': str, 'STATE': str})
df_merge2[['STATE', 'COUNTY', 'TRACT', 'GEOID']]

,STATE,COUNTY,TRACT,GEOID
0,36,061,003001,360610030013002
1,36,061,003001,360610030013002
2,36,061,003001,360610030013002
3,36,061,003001,360610030013002
4,36,061,003001,360610030013002
...,...,...,...,...
2746,36,005,012300,360050123003000
2747,36,005,012300,360050123003000
2748,36,005,012300,360050123003000
2749,36,005,012300,360050123003000


In [ ]:
df_merge2['mergeid']= df_merge2['STATE'] + df_merge2['COUNTY'] + df_merge2['TRACT'] 
df_merge2['mergeid']

0       36061003001
1       36061003001
2       36061003001
3       36061003001
4       36061003001
           ...     
2746    36005012300
2747    36005012300
2748    36005012300
2749    36005012300
2750    36005012300
Name: mergeid, Length: 2751, dtype: object

## 👉 Grab Census Data

1. loading the Census API key

In [ ]:
import dotenv

# Load the environment variables
# (loads CENSUS_API_KEY from .env)
dotenv.load_dotenv()


True

In [ ]:
!touch .env

In [ ]:
%%R 

require('tidycensus')

# because it an environment variable, we don't have to 
# explicitly pass this string to R, it is readable here
# in this R cell.
census_api_key(Sys.getenv("CENSUS_API_KEY"))

To install your API key for use in future sessions, run this function with `install = TRUE`.


2. Decide which Census variables you want

    Use <https://censusreporter.org/> to figure out which tables you want. (if censusreporter is down, check out the code in the cell below)

    -   Scroll to the bottom of the page to see the tables.
    -   If you already know the table ID, stick that in the "Explore" section to learn more about that table.

    By default this code loads (B01003_001) which we found in censusreporter here: https://censusreporter.org/tables/B01003/

    - find some other variables that you're also interested in
    - don't forget to pick a geography like "tract", "county" or "block group". here is the list of [all geographies](https://walker-data.com/tidycensus/articles/basic-usage.html#geography-in-tidycensus
    ).


# Merging with race & income census data 

In [ ]:
%%R -o nyc_census_data

nyc_census_data <- get_acs(geography = "tract", 
                           state = 'NY',
                           county = c("New York", "Kings", "Queens", "Bronx", "Richmond"),
                           variables = c(
                             total = "B03002_001",
                             hispanic = "B03002_012",
                             white_nonhispanic = "B03002_003",
                             black_nonhispanic = "B03002_004",
                             asian_nonhispanic = "B03002_006",
                             native_nonhispanic = "B03002_005",
                             someother_nonhispanic = "B03002_007",
                             two_or_more_nonhispanic = "B03002_008",
                             median_income = "B19013_001"
                          
                           ), 
                           year = 2019,
                           survey = "acs5",
                           geometry = FALSE)

# Display the obtained NYC census data
nyc_census_data

# A tibble: 19,503 × 5
   GEOID       NAME                                   variable    estimate   moe
   <chr>       <chr>                                  <chr>          <dbl> <dbl>
 1 36005000100 Census Tract 1, Bronx County, New York total           6864   331
 2 36005000100 Census Tract 1, Bronx County, New York white_nonh…      603   172
 3 36005000100 Census Tract 1, Bronx County, New York black_nonh…     3601   296
 4 36005000100 Census Tract 1, Bronx County, New York native_non…       18    20
 5 36005000100 Census Tract 1, Bronx County, New York asian_nonh…      181    70
 6 36005000100 Census Tract 1, Bronx County, New York someother_…        0    17
 7 36005000100 Census Tract 1, Bronx County, New York two_or_mor…       19    25
 8 36005000100 Census Tract 1, Bronx County, New York hispanic        2407   255
 9 36005000100 Census Tract 1, Bronx County, New York median_inc…       NA    NA
10 36005000200 Census Tract 2, Bronx County, New York total           4532   471
# ℹ 1

Getting data from the 2015-2019 5-year ACS
Using FIPS code '36' for state 'NY'
Using FIPS code '061' for 'New York County'
Using FIPS code '047' for 'Kings County'
Using FIPS code '081' for 'Queens County'
Using FIPS code '005' for 'Bronx County'
Using FIPS code '085' for 'Richmond County'


In [ ]:
df = nyc_census_data

In [ ]:
df.drop(columns=['moe'], inplace=True)
df

,GEOID,NAME,variable,estimate
1,36005000100,"Census Tract 1, Bronx County, New York",total,6864.0
2,36005000100,"Census Tract 1, Bronx County, New York",white_nonhispanic,603.0
3,36005000100,"Census Tract 1, Bronx County, New York",black_nonhispanic,3601.0
4,36005000100,"Census Tract 1, Bronx County, New York",native_nonhispanic,18.0
5,36005000100,"Census Tract 1, Bronx County, New York",asian_nonhispanic,181.0
...,...,...,...,...
19499,36085990100,"Census Tract 9901, Richmond County, New York",asian_nonhispanic,0.0
19500,36085990100,"Census Tract 9901, Richmond County, New York",someother_nonhispanic,0.0
19501,36085990100,"Census Tract 9901, Richmond County, New York",two_or_more_nonhispanic,0.0
19502,36085990100,"Census Tract 9901, Richmond County, New York",hispanic,0.0


In [ ]:
%%R -i df -o df2

df2 <- df %>% 
pivot_wider(
    names_from=variable, 
    values_from = c(estimate),
    names_glue = "{variable}_{.value}"
  )

df2

# A tibble: 2,167 × 11
   GEOID      NAME  total_estimate white_nonhispanic_es…¹ black_nonhispanic_es…²
   <chr>      <chr>          <dbl>                  <dbl>                  <dbl>
 1 360050001… Cens…           6864                    603                   3601
 2 360050002… Cens…           4532                     33                    920
 3 360050004… Cens…           5516                    387                   1264
 4 360050016… Cens…           5825                    213                   2021
 5 360050019… Cens…           3141                    340                    937
 6 360050020… Cens…           9040                     21                   3031
 7 360050023… Cens…           4302                      1                   1192
 8 360050024… Cens…            155                     65                     90
 9 360050025… Cens…           5610                    220                   1355
10 360050027… Cens…           2608                      0                    643
# ℹ 2

In [ ]:
df_merge2.head()
#36061003001

,Unnamed: 0,Borough,Borough/Citywide Office (B/CO),District,School,School Name_x,School Category,Program,Language,Language (Translated),...,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY,mergeid
0,0,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,-73.986229,3,10664,0,-73.986229,G5040,L,U,061,36061003001
1,1,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,-73.986229,3,10664,0,-73.986229,G5040,L,U,061,36061003001
2,2,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,-73.986229,3,10664,0,-73.986229,G5040,L,U,061,36061003001
3,3,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,-73.986229,3,10664,0,-73.986229,G5040,L,U,061,36061003001
4,4,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,-73.986229,3,10664,0,-73.986229,G5040,L,U,061,36061003001


In [ ]:
df2[df2['GEOID'] == '36061003001']

,GEOID,NAME,total_estimate,white_nonhispanic_estimate,black_nonhispanic_estimate,native_nonhispanic_estimate,asian_nonhispanic_estimate,someother_nonhispanic_estimate,two_or_more_nonhispanic_estimate,hispanic_estimate,median_income_estimate
1130,36061003001,"Census Tract 30.01, New York County, New York",3815.0,2084.0,214.0,0.0,530.0,0.0,0.0,747.0,70057.0


In [ ]:
final_df = df_merge2.merge(df2, left_on='mergeid', right_on='GEOID')

In [ ]:
final_df

,Unnamed: 0,Borough,Borough/Citywide Office (B/CO),District,School,School Name_x,School Category,Program,Language,Language (Translated),...,NAME_y,total_estimate,white_nonhispanic_estimate,black_nonhispanic_estimate,native_nonhispanic_estimate,asian_nonhispanic_estimate,someother_nonhispanic_estimate,two_or_more_nonhispanic_estimate,hispanic_estimate,median_income_estimate
0,0,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,"Census Tract 30.01, New York County, New York",3815.0,2084.0,214.0,0.0,530.0,0.0,0.0,747.0,70057.0
1,1,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,"Census Tract 30.01, New York County, New York",3815.0,2084.0,214.0,0.0,530.0,0.0,0.0,747.0,70057.0
2,2,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,"Census Tract 30.01, New York County, New York",3815.0,2084.0,214.0,0.0,530.0,0.0,0.0,747.0,70057.0
3,3,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,"Census Tract 30.01, New York County, New York",3815.0,2084.0,214.0,0.0,530.0,0.0,0.0,747.0,70057.0
4,4,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,"Census Tract 30.01, New York County, New York",3815.0,2084.0,214.0,0.0,530.0,0.0,0.0,747.0,70057.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,2746,Bronx,District 75,75,75X811,P.S. X811,Secondary School,Transitional Bilingual Education,Spanish,Español,...,"Census Tract 123, Bronx County, New York",4268.0,51.0,1256.0,4.0,18.0,0.0,0.0,2939.0,31519.0
2538,2747,Bronx,District 75,75,75X811,P.S. X811,Secondary School,Transitional Bilingual Education,Spanish,Español,...,"Census Tract 123, Bronx County, New York",4268.0,51.0,1256.0,4.0,18.0,0.0,0.0,2939.0,31519.0
2539,2748,Bronx,District 75,75,75X811,P.S. X811,Secondary School,Transitional Bilingual Education,Spanish,Español,...,"Census Tract 123, Bronx County, New York",4268.0,51.0,1256.0,4.0,18.0,0.0,0.0,2939.0,31519.0
2540,2749,Bronx,District 75,75,75X811,P.S. X811,Secondary School,Transitional Bilingual Education,Spanish,Español,...,"Census Tract 123, Bronx County, New York",4268.0,51.0,1256.0,4.0,18.0,0.0,0.0,2939.0,31519.0


## 👉 Merge it with your data

hint...`tidycensus` provides you data in long format you may need to pivot the census data from long to wide format before merging it with your data

In [ ]:
final_df

,Unnamed: 0,Borough,Borough/Citywide Office (B/CO),District,School,School Name_x,School Category,Program,Language,Language (Translated),...,NAME_y,total_estimate,white_nonhispanic_estimate,black_nonhispanic_estimate,native_nonhispanic_estimate,asian_nonhispanic_estimate,someother_nonhispanic_estimate,two_or_more_nonhispanic_estimate,hispanic_estimate,median_income_estimate
0,0,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,"Census Tract 30.01, New York County, New York",3815.0,2084.0,214.0,0.0,530.0,0.0,0.0,747.0,70057.0
1,1,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,"Census Tract 30.01, New York County, New York",3815.0,2084.0,214.0,0.0,530.0,0.0,0.0,747.0,70057.0
2,2,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,"Census Tract 30.01, New York County, New York",3815.0,2084.0,214.0,0.0,530.0,0.0,0.0,747.0,70057.0
3,3,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,"Census Tract 30.01, New York County, New York",3815.0,2084.0,214.0,0.0,530.0,0.0,0.0,747.0,70057.0
4,4,Manhattan,Manhattan,1,01M020,P.S. 020 Anna Silver,Elementary,Dual Language,Chinese,中文,...,"Census Tract 30.01, New York County, New York",3815.0,2084.0,214.0,0.0,530.0,0.0,0.0,747.0,70057.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,2746,Bronx,District 75,75,75X811,P.S. X811,Secondary School,Transitional Bilingual Education,Spanish,Español,...,"Census Tract 123, Bronx County, New York",4268.0,51.0,1256.0,4.0,18.0,0.0,0.0,2939.0,31519.0
2538,2747,Bronx,District 75,75,75X811,P.S. X811,Secondary School,Transitional Bilingual Education,Spanish,Español,...,"Census Tract 123, Bronx County, New York",4268.0,51.0,1256.0,4.0,18.0,0.0,0.0,2939.0,31519.0
2539,2748,Bronx,District 75,75,75X811,P.S. X811,Secondary School,Transitional Bilingual Education,Spanish,Español,...,"Census Tract 123, Bronx County, New York",4268.0,51.0,1256.0,4.0,18.0,0.0,0.0,2939.0,31519.0
2540,2749,Bronx,District 75,75,75X811,P.S. X811,Secondary School,Transitional Bilingual Education,Spanish,Español,...,"Census Tract 123, Bronx County, New York",4268.0,51.0,1256.0,4.0,18.0,0.0,0.0,2939.0,31519.0


In [ ]:
final_df.to_csv('combinedwithcensus.csv', index=False)